# Web Scraping Basketball Reference Data

I will be using the website Basketball-reference.com in order to scrap data of the players from multiple different seasons. This website provides clean sport data, thus making it easy to scrape the data and create my own database from the data gathered on the website.

Now to import all the libraries we would need in order to begin web scarping the data from the website. 

In [1]:
# import needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

We will now create the function to gather data from the website. It will take in the year we want for that season. 

In [2]:
# function to gather data from the web parameter is the year
def scrape_nba_data(years):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(years)

    try:
        html = urlopen(url)
        soup = BeautifulSoup(html, features="lxml")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers.remove('Rk')

    rows = soup.findAll('tr')[1:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')] 
                    for i in range(len(rows))]
    
    df = pd.DataFrame(rows_data, columns=headers)
    df.dropna(axis = 0, inplace=True)
    df = df.reset_index(drop=True)

    # Replace empty string with '0'
    df.replace('', '0', inplace=True)   

    # convert numeric columns to float
    numeric_columns = df.columns.drop(['Player', 'Pos', 'Tm'])
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    return df

Now that we created the function to be able to gather data let's test it out by looking at the 2020 - 2021 seaon.

In [3]:
# Test to make sure the function works
NBA_season_2020_2021 = scrape_nba_data(2021)
NBA_season_2020_2021

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,PF,21,MIA,61,4,12.1,2.0,3.7,0.544,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,Jaylen Adams,PG,24,MIL,7,0,2.6,0.1,1.1,0.125,...,0.000,0.0,0.4,0.4,0.3,0.0,0.0,0.0,0.1,0.3
2,Steven Adams,C,27,NOP,58,58,27.7,3.3,5.3,0.614,...,0.444,3.7,5.2,8.9,1.9,0.9,0.7,1.3,1.9,7.6
3,Bam Adebayo,C,23,MIA,64,64,33.5,7.1,12.5,0.570,...,0.799,2.2,6.7,9.0,5.4,1.2,1.0,2.6,2.3,18.7
4,LaMarcus Aldridge,C,35,TOT,26,23,25.9,5.4,11.4,0.473,...,0.872,0.7,3.8,4.5,1.9,0.4,1.1,1.0,1.8,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,0.462,...,0.833,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0
701,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,0.559,...,0.628,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1
702,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,0.438,...,0.886,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3
703,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,0.559,...,0.714,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4
